In [ ]:
!pip install biopython scikit-learn --quiet
import pandas as pd
import numpy as np
from Bio import SeqIO
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.metrics import classification_report, accuracy_score
from random import shuffle
def load_sequences(file_paths, window=150, stride=50):
    sequences = []
    labels = []
    for label, path in file_paths.items():
        for record in SeqIO.parse(path, "fasta"):
            seq = str(record.seq).replace("*", "")
            for i in range(0, len(seq) - window + 1, stride):
                frag = seq[i:i+window]
                if len(frag) == window:
                    sequences.append(frag)
                    labels.append(label)
    combined = list(zip(sequences, labels))
    shuffle(combined)
    sequences, labels = zip(*combined)
    return list(sequences), list(labels)
def get_kmers(seq, k=3):
    return [seq[i:i+k] for i in range(len(seq)-k+1)]
def build_kmer_features(sequences, k=3):
    all_kmers = set()
    kmer_counts = []
    for seq in sequences:
        kmers = get_kmers(seq, k)
        count = Counter(kmers)
        kmer_counts.append(count)
        all_kmers.update(count)
    all_kmers = sorted(list(all_kmers))
    df = pd.DataFrame(0, index=np.arange(len(sequences)), columns=all_kmers)
    for i, count in enumerate(kmer_counts):
        for kmer, freq in count.items():
            df.at[i, kmer] = freq
    return df
file_paths = {
    'Zika': '/content/drive/MyDrive/Datasets/Zika_200.fasta',
    'Ebola': '/content/drive/MyDrive/Datasets/Ebola_200.fasta',
    'SARS_CoV_2': '/content/drive/MyDrive/Datasets/SARS_CoV_2_200.fasta',
    'Influenza_A': '/content/drive/MyDrive/Datasets/Influenza_A_200.fasta',
    'Influenza_B': '/content/drive/MyDrive/Datasets/Influenza_B_200.fasta',
    'Tuberculosis': '/content/drive/MyDrive/Datasets/Tuberculosis_200.fasta',
    'Human': '/content/drive/MyDrive/Datasets/Human_200.fasta'
}
data, labels = load_sequences(file_paths, window=150, stride=50)
print(f" Augmented sequences: {len(data)}")

X = build_kmer_features(data, k=3)
y = pd.factorize(pd.Series(labels))[0]

selector = SelectKBest(mutual_info_classif, k=160)
X_selected = selector.fit_transform(X, y)

X_train, X_test, y_train, y_test = train_test_split(
    X_selected, y, test_size=0.2, stratify=y, random_state=42
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

logreg = LogisticRegression(
    penalty='l2',
    C=0.5,
    solver='lbfgs',
    class_weight='balanced',
    max_iter=1000,
    random_state=42
)
logreg.fit(X_train_scaled, y_train)

y_pred = logreg.predict(X_test_scaled)
acc = accuracy_score(y_test, y_pred)

print(f"\n Final Logistic Regression (L2) Accuracy: {acc * 100:.2f}%")
print("\n Classification Report:\n")
print(classification_report(y_test, y_pred))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 20.0 MB/s eta 0:00:00
 Augmented sequences: 9820

 Final Logistic Regression (L2) Accuracy: 95.32%

 Classification Report:

              precision    recall  f1-score   support

           0       0.96      1.00      0.98       829
           1       0.91      0.91      0.91       237
           2       0.98      0.89      0.93       426
           3       0.94      0.97      0.96       151
           4       0.99      0.97      0.98       252
           5       0.77      0.85      0.81        67
           6       0.67      1.00      0.80         2

    accuracy                           0.95      1964
   macro avg       0.89      0.94      0.91      1964
weighted avg       0.95      0.95      0.95      1964



In [ ]:
!pip install biopython scikit-learn --quiet

import pandas as pd
import numpy as np
from Bio import SeqIO
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.metrics import classification_report, accuracy_score
from random import shuffle
def load_sequences(file_paths, window=150, stride=50):
    sequences = []
    labels = []
    for label, path in file_paths.items():
        for record in SeqIO.parse(path, "fasta"):
            seq = str(record.seq).replace("*", "")
            for i in range(0, len(seq) - window + 1, stride):
                frag = seq[i:i+window]
                if len(frag) == window:
                    sequences.append(frag)
                    labels.append(label)
    combined = list(zip(sequences, labels))
    shuffle(combined)
    sequences, labels = zip(*combined)
    return list(sequences), list(labels)
def get_kmers(seq, k=3):
    return [seq[i:i+k] for i in range(len(seq)-k+1)]

def build_kmer_features(sequences, k=3):
    all_kmers = set()
    kmer_counts = []
    for seq in sequences:
        kmers = get_kmers(seq, k)
        count = Counter(kmers)
        kmer_counts.append(count)
        all_kmers.update(count)
    all_kmers = sorted(list(all_kmers))
    df = pd.DataFrame(0, index=np.arange(len(sequences)), columns=all_kmers)
    for i, count in enumerate(kmer_counts):
        for kmer, freq in count.items():
            df.at[i, kmer] = freq
    return df
file_paths = {
    'Zika': '/content/drive/MyDrive/Datasets/Zika_200.fasta',
    'Ebola': '/content/drive/MyDrive/Datasets/Ebola_200.fasta',
    'SARS_CoV_2': '/content/drive/MyDrive/Datasets/SARS_CoV_2_200.fasta',
    'Influenza_A': '/content/drive/MyDrive/Datasets/Influenza_A_200.fasta',
    'Influenza_B': '/content/drive/MyDrive/Datasets/Influenza_B_200.fasta',
    'Tuberculosis': '/content/drive/MyDrive/Datasets/Tuberculosis_200.fasta',
    'Human': '/content/drive/MyDrive/Datasets/Human_200.fasta'
}
data, labels = load_sequences(file_paths, window=150, stride=50)
print(f"\n Augmented sequences: {len(data)}")
X = build_kmer_features(data, k=3)
y = pd.factorize(pd.Series(labels))[0]
selector = SelectKBest(mutual_info_classif, k=160)
X_selected = selector.fit_transform(X, y)
print(X_selected)
selected_indices = selector.get_support(indices=True)
selected_features = X.columns[selected_indices]
X_train, X_test, y_train, y_test = train_test_split(
    X_selected, y, test_size=0.2, stratify=y, random_state=42
)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
logreg = LogisticRegression(
    penalty='l2',
    C=0.5,
    solver='lbfgs',
    class_weight='balanced',
    max_iter=1000,
    random_state=42
)
logreg.fit(X_train_scaled, y_train)
y_pred = logreg.predict(X_test_scaled)
acc = accuracy_score(y_test, y_pred)
print(f"\n Final Logistic Regression (L2) Accuracy: {acc * 100:.2f}%")
print("\n Classification Report:\n")
print(classification_report(y_test, y_pred))
print("\n Selected Features (Top 160 k-mers):")
print(selected_features.tolist())


 Augmented sequences: 9820
[[0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]

 Final Logistic Regression (L2) Accuracy: 95.98%

 Classification Report:

              precision    recall  f1-score   support

           0       0.98      0.87      0.92       426
           1       0.92      0.94      0.93       237
           2       0.97      1.00      0.98       829
           3       1.00      1.00      1.00       252
           4       0.93      0.97      0.95       151
           5       0.83      0.96      0.89        67
           6       1.00      1.00      1.00         2

    accuracy                           0.96      1964
   macro avg       0.95      0.96      0.95      1964
weighted avg       0.96      0.96      0.96      1964


 Selected Features (Top 160 k-mers):
['AAA', 'AAD', 'AAQ', 'AAT', 'AEA', 'AEH', 'AFF', 'AFG', 'AGA', 'AGG', 'ALA', 'ANL', 'APG', 'API', 'ARA', 'ARI', 'ARL', 'ASP', 'ATL', 'ATQ', 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
